# For use in viewing ISR of an image
Craig Lage 16-Feb-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.summit.utils import BestEffortIsr
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    vmin = 10
    vmax = 10000
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return


In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "u/czw/DM-37811/parOStest.20230202a/calib"])
#biasButler = Butler("/repo/embargo", collections=["u/cslage/bias_parallel_13144_B_17jan23"])
#ptcButler = Butler("/repo/embargo", collections=["u/cslage/ptc_bot_cross_parallel_25jan23"])
crossButler = Butler("/repo/embargo", collections=["u/snyder18/20230131/crosstalk_analysis"])
crossButler_run2 = Butler("/repo/embargo", collections=["u/snyder18/20230131/crosstalk_analysis_run2"])

 ### New ISR

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
expId = 2023021500278
exp = butler.get('raw', detector=0, exposure=expId)
biasExp = butler.get('bias', detector=0, exposure=expId)

isrResult = isrTask.run(exp, bias=biasExp)
postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
plt.suptitle(f"{expId}")
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/AuxTel/isr/Streaked_Flat_{expId}_15Feb23.png")